# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Login issues (SaaS)

This notebook contains Kusto queries that can help getting to the root cause of a login issue for an environment in the online version of Business Central (SaaS). Each section in the notebook contains links to the TSG part of the authorization telemetry documentation in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area. 

NB! The signal used in this notebook is only available in version 16.2 (or newer) of Business Central online, so check the version of your environment.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [2]:
# load the KQLmagic module
%reload_ext Kqlmagic

# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'


## 2. Define filters
This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName: 

In [12]:
# aadTenantId = "<Add AAD tenant id here>"
# environmentName = "<add environment name here>"

aadTenantId = "de612f9e-c3a1-4e62-a824-84eff03fd3af"
environmentName = "Production"

# Analyze the login flow
Now you can run Kusto queries to look for possible root causes for login issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

Documentation of the authentication and authorization flow: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application

Kusto sample: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

## Authentication

Authentication in the online version of Business Central happens strictly in Azure Active Directory (AAD). Only when a user is authenticated in AAD, a session is attempted to be created in the Business Central server (NST). When dealing with login issues, check for *absense* of signal in eventIds for Authorization in the pre-open company of opening a session in the NST (eventIds RT0001 and RT0003) to determine if the issue is related to AAD (e.g. user is disabled, wrong password, failed MFA) or maybe something happening in the customer network (could be a DNS issue, or a changed firewall rule). 

**If you do not see any signal for eventIds RT0001 and RT0003, then start troubleshooting network issues first.**

Read more in the Security Guide here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

In [3]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId in ('RT0001', 'RT0003' )
| summarize request_count=count() by bin(timestamp, 1h) | render timechart title= 'Number of pre-open company authorization attempts in the last day'

SyntaxError: invalid syntax (<ipython-input-3-78c387293ff4>, line 5)

## Authorization failures (pre-open company)

A user can fail authorization before the open company trigger is executed for a number of different reasons:
* The user was successfully authenticated in Azure Active Directory but the user account is disabled in Business Central.
* A user successfully authenticated in Azure Active Directory but the user does not have any entitlements in Business Central (license issue)

Read more about these types of failures in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorizationfailedpreopencompany

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [14]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0001'
| project timestamp
, guestUser = customDimensions.guestUser
, userType = customDimensions.userType
, failureReason = customDimensions.failureReason
, entitlementSetIds = customDimensions.entitlementSetIds
| order by timestamp desc
| limit 100

,timestamp,guestUser,userType,failureReason,entitlementSetIds


[]

## Authorization failures (in the open company process)

Events show up here for a number of different reasons
* The company name is invalid
* User has no permission to access the company
* The environment is locked
* The license has expired or the trial period has ended
* The user's license is not valid for use on production companies

Read more in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorization-failed-open-company

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql


In [15]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0002'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, failureReason = customDimensions.failureReason
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,failureReason


[]

## Successful logins (authentication in AAD succeded, authorization in the Business Central server succeeded)

If the user can authenticate against AAD and the two authorization steps inside the Business Central server succeeds, then a session is created and the user has successfully logged in.

Read more about application security in the Security Guide here:  https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

Kusto query: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Authorization.kql

In [16]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0004'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, totalTimeInMS = toreal(totimespan(customDimensions.totalTime))/10000 // totalTime is measured in ticks, divide by 10000 to get milliseconds
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,totalTimeInMS
0,2020-09-03 11:58:45.304946600+00:00,Background,"CRONUS USA, Inc.",4.2552
1,2020-09-03 11:52:42.953805500+00:00,Background,"CRONUS USA, Inc.",4.3144
2,2020-09-03 11:42:06.935431+00:00,Background,"CRONUS USA, Inc.",4.7628
3,2020-09-03 10:53:29.835902100+00:00,Background,"CRONUS USA, Inc.",4.2954
4,2020-09-03 10:12:32.382961900+00:00,Background,"CRONUS USA, Inc.",3.9229
5,2020-09-03 10:01:57.423682300+00:00,Background,"CRONUS USA, Inc.",10.1692
6,2020-09-03 09:39:13.287199800+00:00,Background,"CRONUS USA, Inc.",7.5228
7,2020-09-03 09:25:36.178364400+00:00,Background,"CRONUS USA, Inc.",4.0502
8,2020-09-03 09:24:05.476836500+00:00,Background,"CRONUS USA, Inc.",4.3996
9,2020-09-03 09:22:34.431727600+00:00,Background,"CRONUS USA, Inc.",13.1957


[{'timestamp': datetime.datetime(2020, 9, 3, 11, 58, 45, 304946, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.2552},
 {'timestamp': datetime.datetime(2020, 9, 3, 11, 52, 42, 953805, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.3144},
 {'timestamp': datetime.datetime(2020, 9, 3, 11, 42, 6, 935431, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.7628},
 {'timestamp': datetime.datetime(2020, 9, 3, 10, 53, 29, 835902, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.2954},
 {'timestamp': datetime.datetime(2020, 9, 3, 10, 12, 32, 382961, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 3.9229},
 {'timestamp': datetime.datetime(2020, 9, 3, 10, 1, 57, 423682, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.1692},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 39, 13, 287199, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 7.5228},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 25, 36, 178364, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0502},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 24, 5, 476836, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.3996},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 22, 34, 431727, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 13.1957},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 21, 4, 1971, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 6.618},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 19, 33, 516535, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.156},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 18, 2, 735723, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.1817},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 16, 31, 755434, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0502},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 15, 1, 310163, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.4758},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 13, 30, 803924, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 7.1922},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 12, 0, 66613, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.3704},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 10, 29, 16694, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.7737},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 9, 10, 615809, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0457},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 9, 1, 573133, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.2168},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 8, 58, 570391, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.6628},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 7, 28, 78648, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.9263},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 5, 57, 244591, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.3942},
 {'timestamp': datetime.datetime(2020, 9, 3, 9, 4, 26, 143506, tzinfo=tzutc()), 'clientType': 'Back

In [17]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0004'
    and timestamp > ago(1d)
| extend clientType = tostring( customDimensions.clientType )
| summarize count=count() by clientType, bin(timestamp, 1h)
| render timechart title= 'Number of successful logins the last day (shown by client/session type)'

[{'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 13, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 16, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 17, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 18, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 7, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 8, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 9, 0, tzinfo=tzutc()), 'count': 20},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 19, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 20, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 21, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 22, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 12, 0, tzinfo=tzutc()), 'count': 34},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 23, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 0, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 15, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 4, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 5, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 6, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 10, 0, tzinfo=tzutc()), 'count': 3},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 1, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 2, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 3, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 3, 11, 0, tzinfo=tzutc()), 'count': 3},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 14, 0, tzinfo=tzutc()), 'count': 40}]

In [18]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId in ('RT0001', 'RT0002', 'RT0004')
    and timestamp > ago(1d)
| extend attemptType = case(
    customDimensions.eventId == 'RT0001', 'Failure before open company' ,
    customDimensions.eventId == 'RT0002', 'Failure in open company trigger' ,
    customDimensions.eventId == 'RT0004', 'Successful login' , 
    'Unknown reason'
)
| summarize count=count() by attemptType, bin(timestamp, 1h)
| render timechart title= 'Number of login attempts the last day (shown by success/failure)'

[{'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 12, 0, tzinfo=tzutc()), 'count': 34},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 13, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 16, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 17, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 18, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 19, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 20, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 21, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 23, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 0, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 15, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 7, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 8, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 9, 0, tzinfo=tzutc()), 'count': 20},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 4, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 5, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 6, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 10, 0, tzinfo=tzutc()), 'count': 3},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 1, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 2, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 3, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 22, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 3, 11, 0, tzinfo=tzutc()), 'count': 3},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 14, 0, tzinfo=tzutc()), 'count': 40}]